# Assignment 4: Evaluating Search Engines

For this assignment, we leave aside the code we developed so far, and look into the more general issue of how to evaluate and compare different search engines. The ultimate test for any Information Retrieval system is how well it is able to satisfy the information needs of users.

# Cohen's Kappa

Our evaluation will involve the calculation of [Cohen's kappa](https://en.wikipedia.org/wiki/Cohen's_kappa) to quantify the degree to which two human assessors agree or disagree on whether results are considered relevant or not. To calculate Cohen's kappa, we are going to use the [scikit-learn library](http://scikit-learn.org/stable/):

In [1]:
! pip install --user scikit-learn

In [2]:
from sklearn.metrics import cohen_kappa_score

This library expects relevance assessments as lists of elements where `1` stands for _relevant_ and `0` stands for _not relevant_, for example like this:

In [3]:
a1=[1,0,1,0,1,0,1,0]

This list means that the first document was assessed to be relevant, the second to be not relevant, the third to be relevant etc.

We need two assessments in order to calculate Cohen's kappa, so let's make another exemplary list that only differs on the last element:

In [4]:
a2=[1,0,1,0,1,0,1,1]

We can now invoke the library as follows to calculate the agreement between the two:

In [5]:
cohen_kappa_score(a1, a2)

0.75

This value represents high agreement. We can reach maximal agreement if the two assessments are identical:

In [6]:
cohen_kappa_score(a1, a1)

1.0

Now, let's see what happens for a third assessment that differs on three positions with the first one (the three last positions):

In [7]:
a3=[1,0,1,0,1,1,0,1]

cohen_kappa_score(a1, a3)

0.25

We get a smaller but still positive value, because these two assessments still mostly agree. If we make a further example that differs on 6 of the 8 positions, we get the following result:

In [8]:
a4=[1,0,0,1,0,1,0,1]

cohen_kappa_score(a1, a4)

-0.5

The score is now negative, because the two differ on more positions than they agree. The agreement is in fact less than what you would expect to occur just by chance. We get the maximal disagreement if we define a fifth example that disagrees on all positions:

In [9]:
a5=[0,1,0,1,0,1,0,1]

cohen_kappa_score(a1, a5)

-1.0

Be aware that the kappa score cannot be calculated if you have only `1`s or only `0`s:

In [10]:
a6=[1,1,1,1,1,1,1,1]
a7=[1,1,1,1,1,1,1,1]

cohen_kappa_score(a6, a7)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:638: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


nan

And in the case of a highly skewed set (either vast majority of agreements on `1` or vast majority of agreements on `0`), the kappa score can be counter-intuitive:

In [11]:
a8=[1,1,1,1,1,1,0,1]
a9=[1,1,1,1,1,1,1,0]

cohen_kappa_score(a8, a9)

-0.1428571428571428

Now that we understand how this function works, we will apply it below for our specific evaluation.

# Results and Assessments

Next, we will define some auxilary code to deal with lists of URLs from search engines and associated relevance assessments. We will encode result lists like this:

In [12]:
urls = [
    'https://en.wikipedia.org/wiki/Information_retrieval/',  # 1st result
    'http://www.dictionary.com/browse/information',          # 2nd result
    'https://nlp.stanford.edu/IR-book/'                      # ...
]

And we represent corresponding assessments, as above, as lists of the same size containing relevance values:

In [13]:
my_assessment = [1, 0, 1]
another_assessment = [0, 0, 1]

In order to nicely display URL lists, with or without related assessments, we define a function called `display_results`:

In [14]:
from IPython.display import display, HTML

def display_results(urls, assessment1=None, assessment2=None):
    lines = []
    lines.append('<table>')
    header = '<tr><th>#</th><th>Result URL</th>'
    if (assessment1):
        header += '<th>Assessment 1</th>'
    if (assessment2):
        header += '<th>Assessment 2</th>'
    header += '</tr>'
    lines.append(header)
    i = 0
    for url in urls:
        show_url = url
        if (len(url) > 80):
            show_url = url[:75] + '...'
        line = '<tr><td>{}</td><td><a href="{:s}">{:s}</a></td>'.format(i+1, url, show_url)
        if (assessment1):
            if (assessment1[i] == 0):
                line += '<td><em>Not relevant</em></td>'
            else:
                line += '<td><strong>Relevant</strong></td>'
        if (assessment2):
            if (assessment2[i] == 0):
                line += '<td><em>Not relevant</em></td>'
            else:
                line += '<td><strong>Relevant</strong></td>'
        line += '</tr>'
        lines.append(line)
        i = i+1
    lines.append('</table>')
    display( HTML(''.join(lines)) )

We can use this function to display a list of URLs, optionally together with one or two assessment lists:

In [15]:
print("Just a list of URLs:")
display_results(urls)

print("With one assessment:")
display_results(urls, my_assessment)

print("With two assessments:")
display_results(urls, my_assessment, another_assessment)

Just a list of URLs:


#,Result URL
1,https://en.wikipedia.org/wiki/Information_retrieval/
2,http://www.dictionary.com/browse/information
3,https://nlp.stanford.edu/IR-book/


With one assessment:


#,Result URL,Assessment 1
1,https://en.wikipedia.org/wiki/Information_retrieval/,Relevant
2,http://www.dictionary.com/browse/information,Not relevant
3,https://nlp.stanford.edu/IR-book/,Relevant


With two assessments:


#,Result URL,Assessment 1,Assessment 2
1,https://en.wikipedia.org/wiki/Information_retrieval/,Relevant,Not relevant
2,http://www.dictionary.com/browse/information,Not relevant,Not relevant
3,https://nlp.stanford.edu/IR-book/,Relevant,Relevant


Now we are ready to perform an actual evaluation, which will involve a substantial amount of manual work.

---

# Tasks

**Your name:** Linlin Luo

### Task 1

Think up and formulate a information need (for example in the field of Computer Science or Medicine) for which you think the answer can be found in scientific publications. On page 152 in the book an example of such an information need is shown: "Information on whether drinking red wine is more effective at reducing the risk of heart attacks than white wine."

**Answer:** "Information on whether all orange cats are boy."

Next, write down specifically what documents have to look like to satisfy your information need. For example if your information need is about finding an overview of different cancer types, you could state that a document would need to list at least ten types of cancer to satisfy your information need (among other criteria). Write this down as a protocol with rules and examples. For example, such a protocol could state that at least three out of five given criteria have to be fulfilled for a document to be considered relevant for the information need, and then specify the criteria. Or your protocol could have the form of a sequence of rules, where each rule lets you either label the document as relevant or not relevant, or proceed with the next rule. Such rules and criteria can, for example, be about the general topic of the paper, the concepts mentioned in it, the covered relations between concepts, the type of publication (research paper, overview paper, etc.), the number of references, the types of contained diagrams, and so on, depending on your specified information need.

**Answer:** 
1) A Clear definition of Type 2 diabetes

2) The article mentions the scale of 'excessive' intaking carbohydrates 

3) The article should use data evidence to support the conclusion 

4) The result should include whether excessive intake of carbohydrates relate to the risk of diabetes (Yes or No; not unclear explanation) 

5) The article should mention the example of the exact carbohydrates (eg. food; ripe banana) 

### Task 2

Formulate a keyword query that represents the information need. For the example on page 152 in the book (see above), the example query "wine AND red AND white AND heart AND attack AND effective" is given. (You don't need to use connectors like "AND", but if you do, make first sure your chosen search engines below actually support them.)

**Answer:** orange cats AND boy

Then submit your query to **two** of the following academic search engines:

- [Google Scholar](https://scholar.google.com) (all science disciplines)
- [Semantic Scholar](https://www.semanticscholar.org) (all science disciplines)
- [PubMed Search](https://www.ncbi.nlm.nih.gov/pubmed) (Life Sciences / biomedicine)

The right choice of two from the three search engine depends on the topic of your information need. If your information need is in the Life Sciences and biomedicine, it's probably best to include PubMed Search, but otherwise you should pick Google Scholar and Semantic Scholar.

Extract a list of the top 10 URLs of the lists of each of the search engines given the query. To be ensure that your results are reproducible, it is advised to use the private mode of your browser. Try to access the resulting publications. For the publications where that is not possible (because of dead links or because the publication is pay-walled even within the VU network), exclude them from the list and add more publications to the end of your list (that is, append results number 11, then 12, etc. to ensure you have two lists of 10 publications each). In order to deal with paywalls, you should try accessing the articles from the VU network, use
[UBVU Off-Campus
Access](http://www.ub.vu.nl.vu-nl.idm.oclc.org/nl/faciliteiten/toegang-buiten-de-campus/index.aspx), or try to find the respective documents from alternative sources (Google Scholar, for example, is very good at finding free PDFs of articles). If you get fewer than 10 results for one of the search engines, modify the keyword query above to make it more inclusive, and then redo the steps of this task.

Store your two lists of URLs in the form of Python lists as introduced above. Then, use the `display_results` function to nicely display them.

In [17]:
urls_google = [
    "https://koreascience.kr/article/JAKO200656455785120.page",
    "https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+&btnG=#:~:text=Increased%20consumption%20of%20refined%20carbohydrates%20and%20the%20epidemic%20of%20type%202%20diabetes%20in%20the%20United%20States%3A%20an%20ecologic%20assessment",
    "https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+&btnG=#:~:text=Inadequate%20fat%20or%20carbohydrate%20intake%20was%20associated%20with%20an%20increased%20incidence%20of%20type%202%20diabetes%20mellitus%20in%20Korean%20adults%3A%20a%2012%2Dyear%20community%2Dbased%C2%A0%E2%80%A6",
    "https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+&btnG=#:~:text=The%20effect%20of%20high%20carbohydrate%20diet%20on%20glucose%20tolerance%20in%20patients%20with%20type%202%20diabetes%20mellitus",
    "https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+&btnG=#:~:text=Low%2Dcarbohydrate%20and%20high%2Dfat%20intake%20among%20adult%20patients%20with%20poorly%20controlled%20type%202%20diabetes%20mellitus",
    "https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+&btnG=#:~:text=Dietary%20carbohydrate%20intake%2C%20presence%20of%20obesity%20and%20the%20incident%20risk%20of%20type%202%20diabetes%20in%20Japanese%20men",
    "https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+&btnG=#:~:text=Effect%20of%20high%20protein%20vs%20high%20carbohydrate%20intake%20on%20insulin%20sensitivity%2C%20body%20weight%2C%20hemoglobin%20A1c%2C%20and%20blood%20pressure%20in%20patients%20with%20type%202%20diabetes%C2%A0%E2%80%A6",
    "https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+&btnG=#:~:text=Type%202%20Diabetes%20and%20Dietary%20Carbohydrate%20Intake%20of%20Adolescents%20and%20Young%20Adults%3A%20What%20Is%20the%20Impact%20of%20Different%20Choices%3F",
    "https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+&btnG=#:~:text=Impact%20of%20high%2Dcarbohydrate%20diet%20on%20metabolic%20parameters%20in%20patients%20with%20type%202%20diabetes",
    "https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+&btnG=#:~:text=High%20plasma%20VEGF%20relates%20to%20low%20carbohydrate%20intake%20in%20patients%20with%20type%202%20diabetes"
]

urls_pubmed = [
    "https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+#:~:text=Alternative%20Dietary%20Patterns%20for%20Americans%3A%20Low%2DCarbohydrate%20Diets.",
    "https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+#:~:text=Management%20of%20Type%201%20Diabetes%20With%20a%20Very%20Low%2DCarbohydrate%20Diet.",
    "https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+#:~:text=Dietary%20Patterns%20and%20Growth%2C%20Size%2C%20Body%20Composition%2C%20and/or%20Risk%20of%20Overweight%20or%20Obesity%3A%20A%20Systematic%20Review%20%5BInternet%5D.",
    "https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+#:~:text=Effects%20of%20a%206%2Dmonth%2C%20low%2Dcarbohydrate%20diet%20on%20glycaemic%20control%2C%20body%20composition%2C%20and%20cardiovascular%20risk%20factors%20in%20patients%20with%20type%202%20diabetes%3A%20An%20open%2Dlabel%20randomized%20controlled%20trial.",
    "https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+#:~:text=Interplay%20of%20Dinner%20Timing%20and%20MTNR1B%20Type%202%20Diabetes%20Risk%20Variant%20on%20Glucose%20Tolerance%20and%20Insulin%20Secretion%3A%20A%20Randomized%20Crossover%20Trial.",
    "https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+#:~:text=The%20Okinawan%20diet%3A%20health%20implications%20of%20a%20low%2Dcalorie%2C%20nutrient%2Ddense%2C%20antioxidant%2Drich%20dietary%20pattern%20low%20in%20glycemic%20load.",
    "https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+#:~:text=Association%20between%20dietary%20carbohydrate%20intake%20and%20risk%20of%20type%202%20diabetes%3A%20a%20systematic%20review%20and%20meta%2Danalysis%20of%20cohort%20studies.",
    "https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+#:~:text=Riceabetes%3A%20is%20the%20association%20of%20type%202%20diabetes%20with%20rice%20intake%20due%20to%20a%20high%20carbohydrate%20intake%20or%20due%20to%20exposure%20to%20excess%20inorganic%20arsenic%3F",
    "https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+#:~:text=Dietary%20protein%20intake%20and%20obesity%2Dassociated%20cardiometabolic%20function.",
    "https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND%28%22carbohydrate+intake%22+AND+%22Type+2+diabetes%22%29+AND+%28%22increase*%22+OR+%22elevate*%22+OR+%22relate*%22+OR+%22correlation*%22%29+#:~:text=Diabetes%20in%20India%3A%20what%20is%20different%3F",
]

print("The list of Google URLs:")
display_results(urls_google)
print("The list of PubMed URLs:")
display_results(urls_pubmed)

The list of Google URLs:


#,Result URL
1,https://koreascience.kr/article/JAKO200656455785120.page
2,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...
3,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...
4,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...
5,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...
6,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...
7,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...
8,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...
9,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...
10,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...


The list of PubMed URLs:


#,Result URL
1,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...
2,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...
3,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...
4,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...
5,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...
6,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...
7,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...
8,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...
9,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...
10,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...


### Task 3

Then, find a fellow student who will **independently**
assess the results as "relevant" or "not relevant" using the protocol that you
have defined above, and also help (at least) one other student for his/her
assessment. Write down their names here:

**Name of the student who assesses my results: Kilian Jiang

**Name of the student who I help to assess his/her results: Kilian Jiang

Show to the other assessor everything you have written down above for Tasks 1 and 2 (and you might also want to give him/her the PDFs you got for these papers to simplify the process).

You as assessors need to stick to the protocol you made in Task 1 and should not discuss with each other, especially when you doubt whether a result is relevant or not. Write down your assessments as lists of relevance values, as introduced above, and make sure they correctly map to the URLs by displaying them together with the `display_results` function.

To avoid problems with extreme results, mark in each list at least one paper as 'relevant' and at least one paper as 'not relevant'. That is, if all papers seem relevant, mark the one that seems least relevant 'not relevant', and conversely, if none of the papers seem relevant, mark the one that seems a bit more relevant than the others as 'relevant'.

In [19]:
# Assessment 1 is from you:

assessment1_google = [1, 1, 0, 0, 1, 0, 1, 1, 1, 1]
assessment1_pubmed = [1, 0, 1, 1, 1, 1, 0, 0, 1, 0]

# Assessment 2 is from your fellow student (don't show him/her your own assessment!):

assessment2_google = [1, 1, 0, 1, 1, 0, 1, 1, 1, 1]
assessment2_pubmed = [0, 0, 1, 1, 1, 0, 1, 0, 1, 0]

display_results(urls_google, assessment1_google, assessment2_google)
display_results(urls_pubmed, assessment1_pubmed, assessment2_pubmed)

#,Result URL,Assessment 1,Assessment 2
1,https://koreascience.kr/article/JAKO200656455785120.page,Relevant,Relevant
2,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...,Relevant,Relevant
3,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...,Not relevant,Not relevant
4,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...,Not relevant,Relevant
5,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...,Relevant,Relevant
6,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...,Not relevant,Not relevant
7,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...,Relevant,Relevant
8,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...,Relevant,Relevant
9,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...,Relevant,Relevant
10,https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%28%22excessive%22O...,Relevant,Relevant


#,Result URL,Assessment 1,Assessment 2
1,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...,Relevant,Not relevant
2,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...,Not relevant,Not relevant
3,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...,Relevant,Relevant
4,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...,Relevant,Relevant
5,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...,Relevant,Relevant
6,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...,Relevant,Not relevant
7,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...,Not relevant,Relevant
8,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...,Not relevant,Not relevant
9,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...,Relevant,Relevant
10,https://pubmed.ncbi.nlm.nih.gov/?term=%28%22excessive%22OR+%22High%22%29AND...,Not relevant,Not relevant


### Task 4

Compute Cohen's kappa to quantify how much the two assessors agreed. Use the function `cohen_kappa_score` demonstrated above to calculate two times the inter-annotator agreement (once for each of the two search engines), and print out the results.

In [20]:
kappa_google = cohen_kappa_score(assessment1_google, assessment2_google)
kappa_pubmed = cohen_kappa_score(assessment1_pubmed, assessment2_pubmed)

print("Kappa for Google Scholar:", kappa_google)
print("Kappa for PubMed:", kappa_pubmed)

Kappa for Google Scholar: 0.736842105263158
Kappa for PubMed: 0.4


Explain whether the agreement can be considered high or not, based on the interpretation table on [this Wikipedia page](https://en.wikipedia.org/wiki/Fleiss'_kappa#Interpretation) (this Wikipedia page is about a different type of kappa but the interpretation table can also be used for Cohen's kappa).

**Answer:** According to Wikipedia, in the range of 0.41 to 0,6, Kappa for Google scholar and PubMed show a moderate agreement between raters.

### Task 5

Define a function called `precision_at_n` that calculates Precision@n as described in the lecture slides, which takes as input an assessment list and a value for _n_ and returns the respective Precision@n value. Run this function to calculate Precision@10 (that is, n=10) on all four assessments (two assessors and two search engines).

In [21]:
def precision_at_n(assessment):
    return sum(1 for a in assessment[:10] if a == 1) / 10

# Print out Precision@10 for all assessments here.
print("Precision@10 for assessment1_google: ", precision_at_n(assessment1_google))
print("Precision@10 for assessment1_pubmed: ", precision_at_n(assessment1_pubmed))
print("Precision@10 for assessment2_google: ", precision_at_n(assessment2_google))
print("Precision@10 for assessment2_pubmed: ", precision_at_n(assessment2_pubmed))

Precision@10 for assessment1_google:  0.7
Precision@10 for assessment1_pubmed:  0.6
Precision@10 for assessment2_google:  0.8
Precision@10 for assessment2_pubmed:  0.5


Explain what these specific Precision@10 results tell us (or don't tell us) about the quality of the two search engines for your particular information need. You can also refer to the results of Task 4 if necessary.

**Answer:** In Assessment 1, Google's precision was 70%, while PubMed's precision was 60%. Therefore, Google's results are more relevant. The same results were found in Assessment 2. Google's precision was 80%, while PubMed's precision was only 50%. In both assessments, Google is the absolute winner.

# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done **individually**, and that code sharing or copying are **strictly forbidden** and will be punished.